In [1]:

import pandas as pd

# Creating a matrix of embedders I can use 


In [2]:
# loading in the df 
results_df = pd.read_csv('results_negative.csv')

In [3]:
#loading in dictionary 
# Loading the vocab_dict again
import pickle
with open("updated_vocab_dict.pkl", "rb") as f:
    updated_vocab = pickle.load(f)

In [4]:
len(updated_vocab)

86996

In [5]:
results_df

,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,query,passage_text,negative_sample
0,0,0,0,0,0,0,what is rba,since the rba outstanding reputation been affe...,unaffiliated the majority religion in chile in...
1,1,1,1,1,1,1,what is rba,the reserve bank of rba came into being on as ...,days on average range days for the infectious ...
2,2,2,2,2,2,2,what is rba,rba with the us regional partner of the by pr ...,citrus tree that does not get enough water bec...
3,3,3,3,3,3,3,what is rba,the inner of atomizer are surprisingly simple ...,or are herbal that promote greater density and...
4,4,4,4,4,4,4,what is rba,based accountability also known as rba is way ...,find by star from the big dipper if you re fam...
...,...,...,...,...,...,...,...,...,...
676183,676183,676183,676183,676183,676188,676188,what is polarity index definition,water is an example of polar molecule since it...,size relative to tea cup the porcupine is the ...
676184,676184,676184,676184,676184,676189,676189,what is polarity index definition,supplement can either be polar or nonpolar dep...,average air hostess the average salary for air...
676185,676185,676185,676185,676185,676190,676190,what is polarity index definition,full definition of polarity the quality or con...,one of ranging from to hip hop in june or each...
676186,676186,676186,676186,676186,676191,676191,what is polarity index definition,part of the smart grid glossary also see bipol...,days of hot weather in the temperature at into...


In [6]:
import pandas as pd
import torch
import random
from tqdm import tqdm
import numpy as np  # Import the tqdm library for progress bars

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to create a reverse vocabulary from the updated vocabulary
def create_reverse_vocab(vocab):
    return {word: index for index, word in vocab.items()}

# Load the reverse vocabulary
reverse_vocab = create_reverse_vocab(updated_vocab)

# Function to tokenize a list of texts using the reverse vocabulary
def tokenize_texts(texts, reverse_vocab):
    tokens_list = []
    
    for text in texts:
        words = text.lower().split()  # Convert the text to lowercase to match training preprocessing
        
        tokenized = []
        for word in words:
            if word in reverse_vocab:
                tokenized.append(reverse_vocab[word])  # Get the index from reverse_vocab
            # No else clause needed; unknown words are simply skipped
        
        tokens_list.append(tokenized)
    
    return tokens_list

# Step 1: Shuffle the DataFrame row-wise
shuffled_df = results_df.sample(frac=1).reset_index(drop=True)  # Shuffle the dataframe

# Step 2: Tokenize the query, passage_text, and negative_samples columns
tokenized_queries = tokenize_texts(shuffled_df['query'].tolist(), reverse_vocab)
tokenized_passages = tokenize_texts(shuffled_df['passage_text'].tolist(), reverse_vocab)
tokenized_negatives = tokenize_texts(shuffled_df['negative_sample'].tolist(), reverse_vocab)


In [8]:

# Step 3: Define the SkipGramFoo model
class SkipGramFoo(torch.nn.Module):
    def __init__(self, voc, emb, ctx):
        super().__init__()
        self.ctx = ctx
        self.emb = torch.nn.Embedding(num_embeddings=voc, embedding_dim=emb)
        self.ffw = torch.nn.Linear(in_features=emb, out_features=voc, bias=False)
        self.sig = torch.nn.Sigmoid()

    def forward(self, inpt, trgs, rand):
        emb = self.emb(inpt)
        batch_size = inpt.size(0)
        rand = rand[:batch_size]
        
        ctx = self.ffw.weight[trgs.to(inpt.device)]
        rnd = self.ffw.weight[rand.to(inpt.device)]
        
        out = torch.bmm(ctx.view(batch_size, 1, -1), emb.unsqueeze(2)).squeeze(2)
        rnd = torch.bmm(rnd.view(batch_size, 1, -1), emb.unsqueeze(2)).squeeze(2)

        out = self.sig(out).clamp(min=1e-7, max=1 - 1e-7)
        rnd = self.sig(rnd).clamp(min=1e-7, max=1 - 1e-7)

        pst = -out.log().mean()
        ngt = -(1 - rnd).log().mean()
        
        return pst + ngt

# Load the model
embedding_dim = 64
model_path = "finetuned_skipgram_model.pth"
mFoo = SkipGramFoo(len(updated_vocab), embedding_dim, 2).to(device)
mFoo.load_state_dict(torch.load(model_path), strict=False)
mFoo.eval()

# Step 4: Generate embeddings for tokenized texts (query, passage, negative samples) with a progress bar
def get_embeddings_for_tokens(tokenized_list, model):
    embeddings_list = []
    
    with torch.no_grad():
        for tokens in tqdm(tokenized_list, desc="Generating embeddings", unit="text"):  # Add progress bar
            if len(tokens) > 0:
                token_tensor = torch.LongTensor(tokens).to(device)
                token_embeddings = model.emb(token_tensor)  # Shape: [num_tokens, embedding_dim]
                embeddings_list.append(token_embeddings.cpu().numpy())  # Keep the full sequence embeddings
            else:
                embeddings_list.append(torch.zeros((1, embedding_dim)).cpu().numpy())  # Zero vector for empty sequences
    
    return embeddings_list

query_embeddings = get_embeddings_for_tokens(tokenized_queries, mFoo)

import pickle

# Save the list directly as a pickle file
with open('query_embeddings.pkl', 'wb') as f:
    pickle.dump(query_embeddings, f)



C:\Users\omare\AppData\Local\Temp\ipykernel_27728\1276856253.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mFoo.load_state_dict(torch.load(model_path), strict=False)


In [9]:
passage_embeddings = get_embeddings_for_tokens(tokenized_passages, mFoo)
with open('passage_embeddings.pkl', 'wb') as f:
    pickle.dump(query_embeddings, f)


Generating embeddings: 100%|██████████| 676188/676188 [02:14<00:00, 5041.55text/s]


In [10]:
import numpy as np
negative_embeddings = get_embeddings_for_tokens(tokenized_negatives, mFoo)
with open('negative_embeddings.pkl', 'wb') as f:
    pickle.dump(query_embeddings, f)


Generating embeddings: 100%|██████████| 676188/676188 [02:13<00:00, 5074.11text/s]


In [48]:
results_df.loc[1]

Unnamed: 0.5                                                       1
Unnamed: 0.4                                                       1
Unnamed: 0.3                                                       1
Unnamed: 0.2                                                       1
Unnamed: 0.1                                                       1
Unnamed: 0                                                         1
query                                                    what is rba
passage_text       the reserve bank of rba came into being on as ...
negative_sample    days on average range days for the infectious ...
Name: 1, dtype: object

In [12]:
# indexing embeddings 
first_negative_embedding = query_embeddings[1]  # Get the first embedding

# Check the length of the first embedding
length_of_first_embedding = len(first_negative_embedding)

print("Length of the first negative embedding:", length_of_first_embedding)


Length of the first negative embedding: 4


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence  
from tqdm import tqdm  
import wandb  


class TowerOneRNN(nn.Module):
    def __init__(self):
        super(TowerOneRNN, self).__init__()
        self.rnn = nn.RNN(input_size=64, hidden_size=32, batch_first=True)
        self.fc = nn.Linear(32, 3)

    def forward(self, x, lengths):
        # Pack the padded sequences
        x = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        
        # Pass input through the RNN layer
        x, _ = self.rnn(x)
        
        # Unpack the output and get the last time step
        x, _ = nn.utils.rnn.pad_packed_sequence(x, batch_first=True)
        x = x[torch.arange(x.size(0)), lengths - 1]  # Get the last valid time step
        
        # Pass through the fully connected layer
        x = self.fc(x)
        
        return x

class TowerTwoRNN(nn.Module):
    def __init__(self):
        super(TowerTwoRNN, self).__init__()
        self.rnn = nn.RNN(input_size=64, hidden_size=32, batch_first=True)
        self.fc = nn.Linear(32, 3)

    def forward(self, x, lengths):
        # Pack the padded sequences
        x = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        
        # Pass input through the RNN layer
        x, _ = self.rnn(x)
        
        # Unpack the output and get the last time step
        x, _ = nn.utils.rnn.pad_packed_sequence(x, batch_first=True)
        x = x[torch.arange(x.size(0)), lengths - 1]  # Get the last valid time step
        
        # Pass through the fully connected layer
        x = self.fc(x)
        
        
        return x


In [5]:
import pickle

with open('query_embeddings.pkl', 'rb') as f:
    query_embeddings = pickle.load(f)

with open('passage_embeddings.pkl', 'rb') as f:
    passage_embeddings = pickle.load(f)

with open('negative_embeddings.pkl', 'rb') as f:
    negative_embeddings = pickle.load(f)


In [6]:
len(passage_embeddings)

676188

In [11]:
import numpy as np

wandb.init(project="twotower_training", entity="omareweis123", name='fine_tuning_skipgram/packed_new_skipgram_betterpre')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Tower RNN Models
tower_one = TowerOneRNN().to(device)
tower_two = TowerTwoRNN().to(device)

# Triplet margin loss with cosine distance
triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1.0 - nn.functional.cosine_similarity(x, y),
    margin=1,
    reduction='mean'
)

# Define optimizer for the tower models
optimizer = optim.Adam(list(tower_one.parameters()) + list(tower_two.parameters()), lr=0.001)

# Define number of epochs
num_epochs = 10 # Example

# Track model and hyperparameters in W&B
wandb.watch([tower_one, tower_two], log="all")  # Log gradients and model weights

# Total samples and batch size
total_samples = len(query_embeddings)  # Assuming all arrays have the same length
batch_size = 128  # Number of embeddings to process in a batch

# Training loop
for epoch in range(num_epochs):
    # Shuffle the indices for the embeddings
    indices = np.random.permutation(total_samples)  # Shuffle indices

    for i in tqdm(range(0, total_samples, batch_size), desc=f"Epoch {epoch + 1}/{num_epochs}"):
        # Select the batch of embeddings by index
        batch_indices = indices[i:min(i + batch_size, total_samples)]
        anchor_batch = [query_embeddings[idx] for idx in batch_indices]
        positive_batch = [passage_embeddings[idx] for idx in batch_indices]
        negative_batch = [negative_embeddings[idx] for idx in batch_indices]

        # Pad sequences to the same length
        anchor_batch = pad_sequence([torch.tensor(e) for e in anchor_batch], batch_first=True).to(device)
        positive_batch = pad_sequence([torch.tensor(e) for e in positive_batch], batch_first=True).to(device)
        negative_batch = pad_sequence([torch.tensor(e) for e in negative_batch], batch_first=True).to(device)

        # Get lengths of the original sequences
        anchor_lengths = torch.tensor([len(e) for e in [query_embeddings[idx] for idx in batch_indices]]).cpu()
        positive_lengths = torch.tensor([len(e) for e in [passage_embeddings[idx] for idx in batch_indices]]).cpu()
        negative_lengths = torch.tensor([len(e) for e in [negative_embeddings[idx] for idx in batch_indices]]).cpu()

        # Forward pass through the two towers
        anchor_output = tower_one(anchor_batch, anchor_lengths)
        positive_output = tower_two(positive_batch, positive_lengths)
        negative_output = tower_two(negative_batch, negative_lengths)

        # Calculate triplet loss
        triplet_loss = triplet_loss_fn(anchor_output, positive_output, negative_output)

        # Backpropagation
        optimizer.zero_grad()
        triplet_loss.backward()
        optimizer.step()

        # Log the loss to W&B
        wandb.log({"epoch": epoch + 1, "triplet_loss": triplet_loss.item()})

    print(f"Epoch {epoch + 1}/{num_epochs} completed. Loss: {triplet_loss.item()}")

    wandb.finish()

print("Training complete.")


batch_triplet_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_triplet_loss,1
epoch,1


Epoch 1/5: 100%|██████████| 5283/5283 [02:24<00:00, 36.69it/s]


Epoch 1/5 completed. Loss: 1.0


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
triplet_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,1
triplet_loss,1


Epoch 2/5:   0%|          | 0/5283 [00:00<?, ?it/s]


Error: You must call wandb.init() before wandb.log()

In [17]:
import numpy as np
import torch
import wandb
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

# Initialize W&B
wandb.init(project="twotower_training", entity="omareweis123", name='fine_tuning_skipgram/packed_no_shuffling ')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tower RNN Models
tower_one = TowerOneRNN().to(device)
tower_two = TowerTwoRNN().to(device)

# Triplet margin loss with cosine distance
triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1.0 - nn.functional.cosine_similarity(x, y),
    margin=1.0,
    reduction='mean'
)

# Define optimizer for the tower models
optimizer = optim.Adam(list(tower_one.parameters()) + list(tower_two.parameters()), lr=0.001)

# Define number of epochs
num_epochs = 50  # Example

# Track model and hyperparameters in W&B
wandb.watch([tower_one, tower_two], log="all")  # Log gradients and model weights

# Total samples and batch size
total_samples = len(query_embeddings)  # Assuming all arrays have the same length
batch_size = 128  # Number of embeddings to process in a batch

# Training loop
for epoch in range(num_epochs):
    # Iterate through the dataset without shuffling, in batches
    for i in tqdm(range(0, total_samples, batch_size), desc=f"Epoch {epoch + 1}/{num_epochs}"):
        # Select the batch of embeddings by index
        batch_indices = range(i, min(i + batch_size, total_samples))
        anchor_batch = [query_embeddings[idx] for idx in batch_indices]
        positive_batch = [passage_embeddings[idx] for idx in batch_indices]
        negative_batch = [negative_embeddings[idx] for idx in batch_indices]

        # Pad sequences to the same length
        anchor_batch = pad_sequence([torch.tensor(e) for e in anchor_batch], batch_first=True).to(device)
        positive_batch = pad_sequence([torch.tensor(e) for e in positive_batch], batch_first=True).to(device)
        negative_batch = pad_sequence([torch.tensor(e) for e in negative_batch], batch_first=True).to(device)

        # Get lengths of the original sequences
        anchor_lengths = torch.tensor([len(seq) for seq in anchor_batch]).cpu()
        positive_lengths = torch.tensor([len(seq) for seq in positive_batch]).cpu()
        negative_lengths = torch.tensor([len(seq) for seq in negative_batch]).cpu()

        # Forward pass through the two towers
        anchor_output = tower_one(anchor_batch, anchor_lengths)
        positive_output = tower_two(positive_batch, positive_lengths)
        negative_output = tower_two(negative_batch, negative_lengths)

        # Calculate triplet loss
        triplet_loss = triplet_loss_fn(anchor_output, positive_output, negative_output)

        # Backpropagation
        optimizer.zero_grad()
        triplet_loss.backward()
        optimizer.step()

        # Log the loss to W&B
        wandb.log({"epoch": epoch + 1, "triplet_loss": triplet_loss.item()})

    print(f"Epoch {epoch + 1}/{num_epochs} completed. Loss: {triplet_loss.item()}")

# Finish W&B logging after all epochs
wandb.finish()

print("Training complete.")


Epoch 1/50: 100%|██████████| 5283/5283 [03:09<00:00, 27.95it/s]


Epoch 1/50 completed. Loss: 1.038094401359558


Epoch 2/50: 100%|██████████| 5283/5283 [03:09<00:00, 27.87it/s]


Epoch 2/50 completed. Loss: 1.0059922933578491


Epoch 3/50:  48%|████▊     | 2544/5283 [01:31<01:38, 27.90it/s]


KeyboardInterrupt: 

In [5]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import wandb

# Initialize Weights and Biases (W&B)
wandb.init(project="twotower_training", entity="omareweis123", name='fine_tuning_skipgram/packed_no_shuffling')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tower RNN Models
tower_one = TowerOneRNN().to(device)
tower_two = TowerTwoRNN().to(device)

# Triplet margin loss with cosine distance
triplet_loss_fn = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1.0 - nn.functional.cosine_similarity(x, y),
    margin=1.0,
    reduction='mean'
)

# Define optimizer for the tower models
optimizer = optim.Adam(list(tower_one.parameters()) + list(tower_two.parameters()), lr=0.001)

# Define number of epochs
num_epochs = 5  # Example

# Track model and hyperparameters in W&B
wandb.watch([tower_one, tower_two], log="all")  # Log gradients and model weights

# Ensure the embeddings are converted to lists (if needed)
query_embeddings = [np.array(embedding) for embedding in query_embeddings]
passage_embeddings = [np.array(embedding) for embedding in passage_embeddings]
negative_embeddings = [np.array(embedding) for embedding in negative_embeddings]

# Total samples and batch size
total_samples = len(query_embeddings)  # Assuming all arrays have the same length
batch_size = 64  # Number of embeddings to process in a batch

# Training loop
for epoch in range(num_epochs):
    # Iterate through the dataset without shuffling, in batches
    for i in tqdm(range(0, total_samples, batch_size), desc=f"Epoch {epoch + 1}/{num_epochs}"):
        # Select the batch of embeddings by index
        batch_indices = range(i, min(i + batch_size, total_samples))
        anchor_batch = [query_embeddings[idx] for idx in batch_indices]
        positive_batch = [passage_embeddings[idx] for idx in batch_indices]
        negative_batch = [negative_embeddings[idx] for idx in batch_indices]

        # Pad sequences to the same length
        anchor_batch = pad_sequence([torch.tensor(e) for e in anchor_batch], batch_first=True).to(device)
        positive_batch = pad_sequence([torch.tensor(e) for e in positive_batch], batch_first=True).to(device)
        negative_batch = pad_sequence([torch.tensor(e) for e in negative_batch], batch_first=True).to(device)

        # Get lengths of the original sequences
        anchor_lengths = torch.tensor([len(seq) for seq in anchor_batch]).cpu()
        positive_lengths = torch.tensor([len(seq) for seq in positive_batch]).cpu()
        negative_lengths = torch.tensor([len(seq) for seq in negative_batch]).cpu()

        # Forward pass through the two towers
        anchor_output = tower_one(anchor_batch, anchor_lengths)
        positive_output = tower_two(positive_batch, positive_lengths)
        negative_output = tower_two(negative_batch, negative_lengths)

        # Calculate triplet loss
        triplet_loss = triplet_loss_fn(anchor_output, positive_output, negative_output)

        # Backpropagation
        optimizer.zero_grad()
        triplet_loss.backward()
        optimizer.step()

        # Log the loss to W&B
        wandb.log({"epoch": epoch + 1, "triplet_loss": triplet_loss.item()})

    print(f"Epoch {epoch + 1}/{num_epochs} completed. Loss: {triplet_loss.item()}")

# Finish W&B logging at the end of the training
wandb.finish()

print("Training complete.")


Epoch 1/5: 100%|██████████| 10566/10566 [05:22<00:00, 32.77it/s]


Epoch 1/5 completed. Loss: 1.0010591745376587


Epoch 2/5: 100%|██████████| 10566/10566 [05:20<00:00, 32.92it/s]


Epoch 2/5 completed. Loss: 0.996854841709137


Epoch 3/5: 100%|██████████| 10566/10566 [05:21<00:00, 32.83it/s]


Epoch 3/5 completed. Loss: 1.00316321849823


Epoch 4/5:  87%|████████▋ | 9174/10566 [04:38<00:42, 32.94it/s]


KeyboardInterrupt: 

wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.
